In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gp
from pandas import DataFrame
import math
import seaborn as sns

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Calculate Estimated GDP in Williamsburg
- Total households in three Williamsburg zip codes (11206, 11211, and 11237)
- Average household expenture in Williamsburg
- Spend by category (listed below) that could contribute to GDP (categorical breakouts only available for all of NYC)

In [26]:
# households and average household expenditures for williamsburg zipcodes
# taken from http://www.point2homes.com/US/Neighborhood/NY/Brooklyn/Williamsburg-Demographics.html
households = 68787
total_expend = 40514

In [47]:
# estimate regional GDP using household expenditures as a proxy
gdp = households * total_expend
print 'Estimated Total Household Expenditures in Williamsburg is: $',gdp

Estimated Total Household Expenditures in Williamsburg is: $ 2786836518


In [43]:
# health = .056
# entertain = .042 
# apparel = .039 
# dining = .132 
# other = .058

category_names = ['healthcare', 'entertainment', 'apparel', 'dining','other']
categories = [.056, .042, .039, .132, .058]
my_list = []

for i in categories:
    cat_spend = i*gdp
    my_list.append(cat_spend)
    #print cat_spend

spend = pd.DataFrame(my_list, index= category_names)
spend.rename(columns = {0:'Spend $'}, inplace = True)
spend['Spend %'] = (spend['Spend $']/np.sum(spend['Spend $']))*100
print spend

                    Spend $    Spend %
healthcare     1.560628e+08  17.125382
entertainment  1.170471e+08  12.844037
apparel        1.086866e+08  11.926606
dining         3.678624e+08  40.366972
other          1.616365e+08  17.737003


In [48]:
# total relevant gdp is just the sum of all relevant categories
gdp_est = sum(spend['Spend $'])
print 'Estimated GDP in Williamsburg is: $',int(gdp_est)
print 'Estimated GDP is', (gdp_est/gdp)*100,'% of total estimated expenditures.'

Estimated GDP in Williamsburg is: $ 911295541
Estimated GDP is 32.7 % of total estimated expenditures.


## Calculate the amount of commercial space around the Bedford Ave. station
- use PLUTO data
- Total commercial space for same three Williamsburg zip codes (11206, 11211, 11237)
- Commercial space within radius around the Bedford Ave. station, calculated using ArcGIS

In [62]:
# read in pluto data (taken from Diego's 'Methodology1' notebook)
pluto_df=pd.read_csv('pluto_lehd_blocks.csv')

ct_area=np.sum(pluto_df.comare)
# per Manushi's ArcGIS analysis, we have total Williamsburg commercial square footage
total_comm = 39715979.0

target_area = ct_area/total_comm

print 'Commercial square footage within radius is:', ct_area
print 'Total commercial square footage in Williamsburg is:', int(total_comm)
print 'The percentage of square footage within radius is:', target_area*100,'%'

Commercial square footage within radius is: 2595120
Total commercial square footage in Williamsburg is: 39715979
The percentage of square footage within radius is: 6.53419622364 %


In [60]:
# calculate area gdp
local_gdp = target_area * gdp_est
local_gdp
print 'The estimated size of our economy around the Bedford Station is: $', int(local_gdp)

The estimated size of our economy around the Bedford Station is: $ 59545838


## Calculate impact that shutdown will have on the in-radius economy

In [63]:
# create impact calculation function
# this function returns the estimated monthly decrease in the economy of the impacted area
# current form shows that the impact of the closure is zero in the same month that the closure finishes
def impact_calc(duration, gdp):
    dur_m = duration * 12 # duration in months if we want that
    yr_range = np.arange(1,dur_m+1) # create range of years
    g = 1/(dur_m*2.0)
    loss = [-.5*log((g*i)+.5) for i in yr_range]
    est_gdp = [(1-k)*local_gdp for k in loss]
    impact = [local_gdp - j for j in est_gdp]
    return impact

In [64]:
# run the function to estimate impact of 3-year shutdown
impact_calc(3, local_gdp)

[19821267.706130005,
 19027276.134527467,
 18253910.07079915,
 17500125.013646886,
 16764953.848583095,
 16047499.387062453,
 15346927.783621497,
 14662462.709881283,
 13993380.183331996,
 13339003.964528359,
 12698701.449331477,
 12071879.993647583,
 11457983.617144629,
 10856490.039998606,
 10266908.013091773,
 9688774.9074664563,
 9121654.5333984494,
 8565135.1633326858,
 8018827.7362330034,
 7482364.2237297669,
 6955396.1408798844,
 6437593.1864464432,
 5928641.9994115606,
 5428245.0199993029,
 4936119.4448424205,
 4451996.2671065703,
 3975619.3934148699,
 3506744.830314897,
 3045139.9338181838,
 2590582.7162337005,
 2142861.2051252872,
 1701772.8497590795,
 1267123.9708807766,
 838729.250081487,
 416411.2553826347,
 0.0]